In [1]:
#! /usr/bin/python
import numpy as np
import heapq
from collections import defaultdict
from scipy.stats import binom
from scipy.stats import truncnorm
import math
import random

# Configurable assembly model for simulations
# Author Daniel Mitropolsky, 2018

class Stimulus:
	def __init__(self, k):
		self.k = k

class Area:
	def __init__(self, name, n, k, beta=0.05):
		self.name = name
		self.n = n
		self.k = k
		# Default beta
		self.beta = beta
		# Betas from stimuli into this area.
		self.stimulus_beta = {}
		# Betas form areas into this area.
		self.area_beta = {}
		self.w = 0
		# List of winners currently (after previous action). Can be 
		# read by caller.
		self.winners = []
		self.new_w = 0
		# new winners computed DURING a projection, do not use outside of internal project function
		self.new_winners = []
		# list of lists of all winners in each round
		self.saved_winners = []
		# list of size of support in each round
		self.saved_w = []
		self.num_first_winners = -1

	def update_winners(self):
		self.winners = self.new_winners
		self.w = self.new_w

	def update_stimulus_beta(self, name, new_beta):
		self.stimulus_beta[name] = new_beta

	def update_area_beta(self, name, new_beta):
		self.area_beta[name] = new_beta

class Brain:
	def __init__(self, p, save_size=True, save_winners=False):
		self.areas = {}
		self.stimuli = {}
		self.stimuli_connectomes = {}
		self.connectomes = {} 
		self.p = p
		self.save_size = save_size
		self.save_winners = save_winners

	def add_stimulus(self, name, k):
		self.stimuli[name] = Stimulus(k)
		new_connectomes = {}
		for key in self.areas:
			new_connectomes[key] = np.empty((0,0))
			self.areas[key].stimulus_beta[name] = self.areas[key].beta
		self.stimuli_connectomes[name] = new_connectomes

	def add_area(self, name, n, k, beta):
		self.areas[name] = Area(name, n, k, beta)

		for stim_name,stim_connectomes in self.stimuli_connectomes.iteritems():
			stim_connectomes[name] = np.empty(0)
			self.areas[name].stimulus_beta[stim_name] = beta

		new_connectomes = {}
		for key in self.areas:
			new_connectomes[key] = np.empty((0,0))
			if key != name:
				self.connectomes[key][name] = np.empty((0,0))
			self.areas[key].area_beta[name] = self.areas[key].beta
			self.areas[name].area_beta[key] = beta
		self.connectomes[name] = new_connectomes

	def update_plasticities(self, area_update_map={}, stim_update_map={}):
		# area_update_map consists of area1: list[ (area2, new_beta) ]
		# represents new plasticity FROM area2 INTO area1
		for to_area, update_rules in area_update_map.items():
			for (from_area, new_beta) in update_rules: 
				self.areas[to_area].area_beta[from_area] = new_beta

		# stim_update_map consists of area: list[ (stim, new_beta) ]f
		# represents new plasticity FROM stim INTO area
		for area, update_rules in stim_update_map.items():
			for (stim, new_beta) in update_rules:
				self.areas[area].stimulus_beta[stim] = new_beta

	def project(self, stim_to_area, area_to_area, verbose=False):
		# Validate stim_area, area_area well defined
		# stim_to_area: {"stim1":["A"], "stim2":["C","A"]}
		# area_to_area: {"A":["A","B"],"C":["C","A"]}

		stim_in = defaultdict(lambda: [])
		area_in = defaultdict(lambda: [])

		for stim, areas in stim_to_area.iteritems():
			if stim not in self.stimuli:
				raise IndexError(stim + " not in brain.stimuli")
				return
			for area in areas:
				if area not in self.areas:
					raise IndexError(area + " not in brain.areas")
					return
				stim_in[area].append(stim)
		for from_area, to_areas in area_to_area.iteritems():
			if from_area not in self.areas:
				raise IndexError(from_area + " not in brain.areas")
				return
			for to_area in to_areas:
				if to_area not in self.areas:
					raise IndexError(to_area + " not in brain.areas")
					return
				area_in[to_area].append(from_area)

		to_update = set().union(stim_in.keys(), area_in.keys())

		for area in to_update:
			num_first_winners = self.project_into(self.areas[area], stim_in[area], area_in[area], verbose)
			self.areas[area].num_first_winners = num_first_winners
			if self.save_winners:
				self.areas[area].saved_winners.append(self.areas[area].new_winners)

		# once done everything, for each area in to_update: area.update_winners()
		for area in to_update:
			self.areas[area].update_winners()
			if self.save_size:
				self.areas[area].saved_w.append(self.areas[area].w)

	def project_into(self, area, from_stimuli, from_areas, verbose=False):
	# projecting everything in from stim_in[area] and area_in[area]
	# calculate: inputs to self.connectomes[area] (previous winners)
	# calculate: potential new winners, Binomial(sum of in sizes, k-top)
	# k top of previous winners and potential new winners
	# if new winners > 0, redo connectome and intra_connectomes 
	# have to wait to replace new_winners
		print("Projecting " + ",".join(from_stimuli) + " and " + ",".join(from_areas) + " into " + area.name)

		name = area.name
		prev_winner_inputs = [0.] * area.w
		for stim in from_stimuli:
			stim_inputs = self.stimuli_connectomes[stim][name]
			for i in xrange(area.w):
				prev_winner_inputs[i] += stim_inputs[i]
		for from_area in from_areas:
			connectome = self.connectomes[from_area][name]
			for w in self.areas[from_area].winners:
				for i in xrange(area.w):
					prev_winner_inputs[i] += connectome[w][i]

		if verbose:
			print ("prev_winner_inputs: ")
			print (prev_winner_inputs)

		# simulate area.k potential new winners
		total_k = 0
		input_sizes = []
		num_inputs = 0
		for stim in from_stimuli:
			total_k += self.stimuli[stim].k
			input_sizes.append(self.stimuli[stim].k)
			num_inputs += 1
		for from_area in from_areas:
			#if self.areas[from_area].w < self.areas[from_area].k:
			#	raise ValueError("Area " + from_area + "does not have enough support.")
			effective_k = len(self.areas[from_area].winners)
			total_k += effective_k
			input_sizes.append(effective_k)
			num_inputs += 1

		if verbose:
			print ("total_k = " + str(total_k) + " and input_sizes = " + str(input_sizes))

		effective_n = area.n - area.w
		# Threshold for inputs that are above (n-k)/n percentile.
		# self.p can be changed to have a custom connectivity into thi sbrain area.
		alpha = binom.ppf((float(effective_n-area.k)/effective_n), total_k, self.p)
		if verbose:
			print("Alpha = " + str(alpha))
		# use normal approximation, between alpha and total_k, round to integer
		# create k potential_new_winners
		std = math.sqrt(total_k * self.p * (1.0-self.p))
		mu = total_k * self.p
		a = float(alpha - mu) / std
		b = float(total_k - mu) / std
		potential_new_winners = truncnorm.rvs(a, b, scale=std, size=area.k)
		for i in xrange(area.k):
			potential_new_winners[i] += mu
			potential_new_winners[i] = round(potential_new_winners[i])
		potential_new_winners = potential_new_winners.tolist()

		if verbose:
			print ("potential_new_winners: ")
			print (potential_new_winners)

		# take max among prev_winner_inputs, potential_new_winners
		# get num_first_winners (think something small)
		# can generate area.new_winners, note the new indices
		both = prev_winner_inputs + potential_new_winners
		new_winner_indices = heapq.nlargest(area.k, range(len(both)), both.__getitem__)
		num_first_winners = 0
		first_winner_inputs = []
		for i in xrange(area.k):
			if new_winner_indices[i] >= area.w:
				first_winner_inputs.append(potential_new_winners[new_winner_indices[i] - area.w])
				new_winner_indices[i] = area.w+ num_first_winners
				num_first_winners += 1
		area.new_winners = new_winner_indices
		area.new_w = area.w + num_first_winners

		# print name + " num_first_winners = " + str(num_first_winners)

		if verbose:
			print ("new_winners: ")
			print (area.new_winners)

		# for i in num_first_winners
		# generate where input came from
			# 1) can sample input from array of size total_k, use ranges
			# 2) can use stars/stripes method: if m total inputs, sample (m-1) out of total_k
		first_winner_to_inputs = {}
		for i in xrange(num_first_winners):
			input_indices = random.sample(xrange(0, total_k), int(first_winner_inputs[i]))
			inputs = np.zeros(num_inputs)
			total_so_far = 0
			for j in xrange(num_inputs):
				inputs[j] = sum([((total_so_far + input_sizes[j]) > w >= total_so_far) for w in input_indices])
				total_so_far += input_sizes[j]
			first_winner_to_inputs[i] = inputs
			if verbose:
				print ("for first_winner # " + str(i) + " with input " + str(first_winner_inputs[i]) + " split as so: ")
				print (inputs)

		m = 0
		# connectome for each stim->area
			# add num_first_winners cells, sampled input * (1+beta)
			# for i in repeat_winners, stimulus_inputs[i] *= (1+beta)
		for stim in from_stimuli:
			if num_first_winners > 0:
				self.stimuli_connectomes[stim][name] = np.resize(self.stimuli_connectomes[stim][name],
					area.w + num_first_winners)
			for i in xrange(num_first_winners):
				self.stimuli_connectomes[stim][name][area.w + i] = first_winner_to_inputs[i][m]
			stim_to_area_beta = area.stimulus_beta[stim]
			for i in area.new_winners:
				self.stimuli_connectomes[stim][name][i] *= (1+stim_to_area_beta)
			if verbose:
				print (stim + " now looks like: " )
				print( self.stimuli_connectomes[stim][name])
			m += 1

		# connectome for each in_area->area
			# add num_first_winners columns
			# for each i in num_first_winners, fill in (1+beta) for chosen neurons
			# for each i in repeat_winners, for j in in_area.winners, connectome[j][i] *= (1+beta)
		for from_area in from_areas:
			from_area_w = self.areas[from_area].w
			from_area_winners = self.areas[from_area].winners
			self.connectomes[from_area][name] = np.pad(self.connectomes[from_area][name], 
				((0,0),(0,num_first_winners)), 'constant', constant_values=0)
			for i in xrange(num_first_winners):
				total_in = first_winner_to_inputs[i][m]
				sample_indices = random.sample(from_area_winners, int(total_in))
				for j in xrange(from_area_w):
					if j in sample_indices:
						self.connectomes[from_area][name][j][area.w+i] = 1
					if j not in from_area_winners:
						self.connectomes[from_area][name][j][area.w+i] = np.random.binomial(1,self.p)
			area_to_area_beta = area.area_beta[from_area]
			for i in area.new_winners:
				for j in from_area_winners:
					self.connectomes[from_area][name][j][i] *= (1.0 +area_to_area_beta)
			if verbose:
				print ("Connectome of " + from_area + " to " + name + " is now:")
				print (self.connectomes[from_area][name])
			m += 1

		# expand connectomes from other areas that did not fire into area
		# also expand connectome for area->other_area
		for other_area in self.areas:
			if other_area not in from_areas:
				self.connectomes[other_area][name] = np.pad(self.connectomes[other_area][name], 
					((0,0),(0,num_first_winners)), 'constant', constant_values=0)
				for j in xrange(self.areas[other_area].w):
					for i in xrange(area.w, area.new_w):
						self.connectomes[other_area][name][j][i] = np.random.binomial(1,self.p)
			# add num_first_winners rows, all bernoulli with probability p
			self.connectomes[name][other_area] = np.pad(self.connectomes[name][other_area],
				((0, num_first_winners),(0, 0)), 'constant', constant_values=0)
			columns = len(self.connectomes[name][other_area][0])
			for i in xrange(area.w, area.new_w):
				for j in xrange(columns):
					self.connectomes[name][other_area][i][j] = np.random.binomial(1,self.p)
			if verbose:
				print ("Connectome of " + name + " to " + other_area + " is now:")
				print (self.connectomes[name][other_area])

		return num_first_winners

In [2]:
!pip install brain

     |████████████████████████████████| 81kB 4.1MB/s 
  Created wheel for brain: filename=brain-0.1.6-cp36-none-any.whl size=82164 sha256=d71bb4de0fb322f3baca7d143d5fd3d2354b495d474e976326885fdc35b25c48
  Stored in directory: /root/.cache/pip/wheels/60/6c/c3/6f636f985898f7b252d1d5a53b5def97e58e6e0596f6793397
Successfully built brain


In [3]:
import brain
import numpy as np
import random
import copy
import pickle
import matplotlib.pyplot as plt

from collections import OrderedDict

# Save obj (could be Brain object, list of saved winners, etc) as file_name
def sim_save(file_name, obj):
	with open(file_name,'wb') as f:
		pickle.dump(obj, f)

def sim_load(file_name):
	with open(file_name,'rb') as f:
		return pickle.load(f)

# Compute item overlap between two lists viewed as sets.
def overlap(a,b):
	return len(set(a) & set(b))

# Compute overlap of each list of winners in winners_list 
# with respect to a specific winners set, namely winners_list[base]
def get_overlaps(winners_list,base,percentage=False):
	overlaps = []
	base_winners = winners_list[base]
	k = len(base_winners)
	for i in xrange(len(winners_list)):
		o = overlap(winners_list[i],base_winners)
		if percentage:
			overlaps.append(float(o)/float(k))
		else:
			overlaps.append(o)
	return overlaps

In [4]:
!pip install brain_util

ERROR: Could not find a version that satisfies the requirement brain_util (from versions: none)
ERROR: No matching distribution found for brain_util


In [5]:
# Library of simulations for preservation overlap in assemblies.
# Assembly operations preserve overlap; the larger the overlap
# between two assemblies x,y in area A, the larger we expect the overlap of 
# proj(x,B) and proj(y,B) to be.

# In our efficiently computable sampling-trick simulation, it is not obvious
# how to create the original assemblies x,y with some particular % overlap
# because we do not model the entire brain area (i.e. sampling-trick: we just model
# the amount needed to simulate projections).
# Hence, we use another property of assemblies: association. If assembly x in A and 
# assembly y in B are fired together into C, proj(x,A) and proj(y,C) will have
# more overlap the more x,y were fired together.
# Hence we exhibit overlap preservation in the following procedure: 
# Create assembly x in A
# Create assembly y in B
# proj(x,C) (until stability)
# proj(y,C) (until stability)
# at this point proj(x,C) and proj(y,C) have as much overlap as random chance (i.e. p)
# associate(x,y,C)
# Now proj(x,C) and proj(y,C) have some percentage overlap
# Now with 4th area D: proj(proj(x,C),D) and proj(proj(y,C),D) for 1 time step

import brain
import numpy as np
import copy

def overlap_sim(n=100000,k=317,p=0.05,beta=0.1,project_iter=10):
	b = brain.Brain(p,save_winners=True)
	b.add_stimulus("stimA",k)
	b.add_area("A",n,k,beta)
	b.add_stimulus("stimB",k)
	b.add_area("B",n,k,beta)
	b.add_area("C",n,k,beta)
	b.add_area("D",n,k,0.0) # final project test area
	b.project({"stimA":["A"],"stimB":["B"]},{})
	# Create assemblies A and B to stability
	for i in xrange(9):
		b.project({"stimA":["A"],"stimB":["B"]},
			{"A":["A"],"B":["B"]})
	b.project({"stimA":["A"]},{"A":["A","C"]})
	# Project A->C
	for i in xrange(9):
		b.project({"stimA":["A"]},
			{"A":["A","C"],"C":["C"]})
	# Project B->C
	b.project({"stimB":["B"]},{"B":["B","C"]})
	for i in xrange(9):
		b.project({"stimB":["B"]},
			{"B":["B","C"],"C":["C"]})
	# Project both A,B to C
	b.project({"stimA":["A"],"stimB":["B"]},
		{"A":["A","C"],"B":["B","C"]})
	for i in xrange(project_iter):
		b.project({"stimA":["A"],"stimB":["B"]},
				{"A":["A","C"],"B":["B","C"],"C":["C"]})
	# Project just B
	b.project({"stimB":["B"]},{"B":["B","C"]})
	# compute overlap
	intersection = bu.overlap(b.areas["C"].saved_winners[-1],b.areas["C"].saved_winners[9])
	assembly_overlap = float(intersection)/float(k)

	b.project({},{"C":["D"]})
	# Project just A
	b.project({"stimA":["A"]},{"A":["A","C"]})
	b.project({},{"C":["D"]})
	D_saved_winners = b.areas["D"].saved_winners
	proj_intersection = bu.overlap(D_saved_winners[0], D_saved_winners[1])
	proj_overlap = float(proj_intersection)/float(k)

	return assembly_overlap, proj_overlap

def overlap_grand_sim(n=100000,k=317,p=0.01,beta=0.05,min_iter=10,max_iter=30):
	b = brain.Brain(p,save_winners=True)
	b.add_stimulus("stimA",k)
	b.add_area("A",n,k,beta)
	b.add_stimulus("stimB",k)
	b.add_area("B",n,k,beta)
	b.add_area("C",n,k,beta)
	b.add_area("D",n,k,0)

	b.project({"stimA":["A"],"stimB":["B"]},{})
	# Create assemblies A and B to stability
	for i in xrange(10):
		b.project({"stimA":["A"],"stimB":["B"]},
			{"A":["A"],"B":["B"]})
	b.project({"stimA":["A"]},{"A":["A","C"]})
	# Project A->C
	for i in xrange(10):
		b.project({"stimA":["A"]},
			{"A":["A","C"],"C":["C"]})
	# Project B->C
	b.project({"stimB":["B"]},{"B":["B","C"]})
	for i in xrange(10):
		b.project({"stimB":["B"]},
			{"B":["B","C"],"C":["C"]})
	# Project both A,B to C
	b.project({"stimA":["A"],"stimB":["B"]},
		{"A":["A","C"],"B":["B","C"]})
	for i in xrange(min_iter-2):
		b.project({"stimA":["A"],"stimB":["B"]},
				{"A":["A","C"],"B":["B","C"],"C":["C"]})
	results = {}
	for i in xrange(min_iter,max_iter+1):
		b.project({"stimA":["A"],"stimB":["B"]},
				{"A":["A","C"],"B":["B","C"],"C":["C"]})
		b_copy1 = copy.deepcopy(b)
		b_copy2 = copy.deepcopy(b)
		# in copy 1, project just A
		b_copy1.project({"stimA":["A"]},{})
		b_copy1.project({},{"A":["C"]})
		# in copy 2, project just B
		b_copy2.project({"stimB":["B"]},{})
		b_copy2.project({},{"B":["C"]})
		intersection = bu.overlap(b_copy1.areas["C"].winners, b_copy2.areas["C"].winners)
		assembly_overlap = float(intersection)/float(k)

		# projecting into D
		b_copy1.project({},{"C":["D"]})
		b_copy1.project({"stimB":["B"]},{})
		b_copy1.project({},{"B":["C"]})
		b_copy1.project({},{"C":["D"]})
		D_saved_winners = b_copy1.areas["D"].saved_winners
		proj_intersection = bu.overlap(D_saved_winners[0], D_saved_winners[1])
		proj_overlap = float(proj_intersection)/float(k)

		print ("t=" + str(i) + " : " + str(assembly_overlap) + " -> " + str(proj_overlap) + "\n")
  
		results[assembly_overlap] = proj_overlap
	return results

In [6]:
# Default simulation library containing:
# - Basic projection simulations (convergence for different beta, etc)
# - Merge simulations (different betas)
# - Pattern completion simulations
# - Association simulations
# - simulations studying density in assemblies (higher than ambient p)

# Also contains methods for plotting saved results from some of these simulations
# (for figures).

import brain

import numpy as np
import random
import copy
import pickle
import matplotlib.pyplot as plt

from collections import OrderedDict


def project_sim(n=1000000,k=1000,p=0.01,beta=0.05,t=50):
	b = brain.Brain(p)
	b.add_stimulus("stim",k)
	b.add_area("A",n,k,beta)
	b.project({"stim":["A"]},{})
	for i in xrange(t-1):
		b.project({"stim":["A"]},{"A":["A"]})
	return b.areas["A"].saved_w


def project_beta_sim(n=100000,k=317,p=0.01,t=100):
	results = {}
	for beta in [0.25,0.1,0.075,0.05,0.03,0.01,0.007,0.005,0.003,0.001]:
		print ("Working on " + str(beta) + "\n")
		out = project_sim(n,k,p,beta,t)
		results[beta] = out
	return results

def assembly_only_sim(n=100000,k=317,p=0.05,beta=0.05,project_iter=10):
	b = brain.Brain(p)
	b.add_stimulus("stim",k)
	b.add_area("A",n,k,beta)
	b.project({"stim":["A"]},{})
	for i in xrange(project_iter-1):
		b.project({"stim":["A"]},{"A":["A"]})
	for i in xrange(5):
		b.project({},{"A":["A"]})
	return b.areas["A"].saved_w


# alpha = percentage of (random) final assembly neurons to try firing
def pattern_com(n=100000,k=317,p=0.05,beta=0.05,project_iter=10,alpha=0.5,comp_iter=1):
	b = brain.Brain(p,save_winners=True)
	b.add_stimulus("stim",k)
	b.add_area("A",n,k,beta)
	b.project({"stim":["A"]},{})
	for i in xrange(project_iter-1):
		b.project({"stim":["A"]},{"A":["A"]})
	# pick random subset of the neurons to fire
	subsample_size = int(k*alpha)
	subsample = random.sample(b.areas["A"].winners, subsample_size)
	b.areas["A"].winners = subsample
	for i in xrange(comp_iter):
		b.project({},{"A":["A"]})
	return b.areas["A"].saved_w,b.areas["A"].saved_winners

def pattern_com_repeated(n=100000,k=317,p=0.05,beta=0.05,project_iter=12,alpha=0.4,
	trials=3, max_recurrent_iter=10, resample=False):
	b = brain.Brain(p,save_winners=True)
	b.add_stimulus("stim",k)
	b.add_area("A",n,k,beta)
	b.project({"stim":["A"]},{})
	for i in xrange(project_iter-1):
		b.project({"stim":["A"]},{"A":["A"]})

	subsample_size = int(k*alpha)
	rounds_to_completion = []
	# pick random subset of the neurons to fire
	subsample = random.sample(b.areas["A"].winners, subsample_size)
	for trail in xrange(trials):
		if resample:
			subsample = random.sample(b.areas["A"].winners, subsample_size)
		b.areas["A"].winners = subsample
		rounds = 0
		while True:
			rounds += 1
			b.project({},{"A":["A"]})
			if (b.areas["A"].num_first_winners == 0) or (rounds == max_recurrent_iter):
				break
		rounds_to_completion.append(rounds)
	saved_winners = b.areas["A"].saved_winners
	overlaps = bu.get_overlaps(saved_winners,project_iter-1,percentage=True)
	return overlaps, rounds_to_completion

def pattern_com_alphas(n=100000,k=317,p=0.01,beta=0.05,
	alphas=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],project_iter=25,comp_iter=5):
	b = brain.Brain(p)
	b.add_stimulus("stim",k)
	b.add_area("A",n,k,beta)
	b.project({"stim":["A"]},{})
	for i in xrange(project_iter-1):
		b.project({"stim":["A"]},{"A":["A"]})
	results = {}
	A_winners = b.areas["A"].winners
	for alpha in alphas:
		# pick random subset of the neurons to fire
		subsample_size = int(k*alpha)
		b_copy = copy.deepcopy(b)
		subsample = random.sample(b_copy.areas["A"].winners, subsample_size)
		b_copy.areas["A"].winners = subsample
		for i in xrange(comp_iter):
			b_copy.project({},{"A":["A"]})
		final_winners = b_copy.areas["A"].winners
		o = bu.overlap(final_winners, A_winners)
		results[alpha] = float(o)/float(k)
	return results

def pattern_com_iterations(n=100000,k=317,p=0.01,beta=0.05,alpha=0.4,comp_iter=8,
	min_iter=20,max_iter=30):
	b = brain.Brain(p)
	b.add_stimulus("stim",k)
	b.add_area("A",n,k,beta)
	b.project({"stim":["A"]},{})
	for i in xrange(min_iter-2):
		b.project({"stim":["A"]},{"A":["A"]})
	results = {}
	subsample_size = int(k*alpha)
	subsample = random.sample(b.areas["A"].winners, subsample_size)
	for i in xrange(min_iter,max_iter+1):
		b.project({"stim":["A"]},{"A":["A"]})
		b_copy = copy.deepcopy(b)
		b_copy.areas["A"].winners = subsample
		for j in xrange(comp_iter):
			b_copy.project({},{"A":["A"]})
		o = bu.overlap(b_copy.areas["A"].winners, b.areas["A"].winners)
		results[i] = float(o)/float(k)
	return results

# Sample command c_w,c_winners = bu.association_sim()
def associate(n=100000,k=317,p=0.05,beta=0.1,overlap_iter=10):
	b = brain.Brain(p,save_winners=True)
	b.add_stimulus("stimA",k)
	b.add_area("A",n,k,beta)
	b.add_stimulus("stimB",k)
	b.add_area("B",n,k,beta)
	b.add_area("C",n,k,beta)
	b.project({"stimA":["A"],"stimB":["B"]},{})
	# Create assemblies A and B to stability
	for i in xrange(9):
		b.project({"stimA":["A"],"stimB":["B"]},
			{"A":["A"],"B":["B"]})
	b.project({"stimA":["A"]},{"A":["A","C"]})
	# Project A->C
	for i in xrange(9):
		b.project({"stimA":["A"]},
			{"A":["A","C"],"C":["C"]})
	# Project B->C
	b.project({"stimB":["B"]},{"B":["B","C"]})
	for i in xrange(9):
		b.project({"stimB":["B"]},
			{"B":["B","C"],"C":["C"]})
	# Project both A,B to C
	b.project({"stimA":["A"],"stimB":["B"]},
		{"A":["A","C"],"B":["B","C"]})
	for i in xrange(overlap_iter-1):
		b.project({"stimA":["A"],"stimB":["B"]},
				{"A":["A","C"],"B":["B","C"],"C":["C"]})
	# Project just B
	b.project({"stimB":["B"]},{"B":["B","C"]})
	for i in xrange(9):
		b.project({"stimB":["B"]},{"B":["B","C"],"C":["C"]})
	return b

def association_sim(n=100000,k=317,p=0.05,beta=0.1,overlap_iter=10):
	b = associate(n,k,p,beta,overlap_iter)
	return b.areas["C"].saved_w,b.areas["C"].saved_winners

def association_grand_sim(n=100000,k=317,p=0.01,beta=0.05,min_iter=10,max_iter=20):
	b = brain.Brain(p,save_winners=True)
	b.add_stimulus("stimA",k)
	b.add_area("A",n,k,beta)
	b.add_stimulus("stimB",k)
	b.add_area("B",n,k,beta)
	b.add_area("C",n,k,beta)
	b.project({"stimA":["A"],"stimB":["B"]},{})
	# Create assemblies A and B to stability
	for i in xrange(9):
		b.project({"stimA":["A"],"stimB":["B"]},
			{"A":["A"],"B":["B"]})
	b.project({"stimA":["A"]},{"A":["A","C"]})
	# Project A->C
	for i in xrange(9):
		b.project({"stimA":["A"]},
			{"A":["A","C"],"C":["C"]})
	# Project B->C
	b.project({"stimB":["B"]},{"B":["B","C"]})
	for i in xrange(9):
		b.project({"stimB":["B"]},
			{"B":["B","C"],"C":["C"]})
	# Project both A,B to C
	b.project({"stimA":["A"],"stimB":["B"]},
		{"A":["A","C"],"B":["B","C"]})
	for i in xrange(min_iter-2):
		b.project({"stimA":["A"],"stimB":["B"]},
				{"A":["A","C"],"B":["B","C"],"C":["C"]})
	results = {}
	for i in xrange(min_iter,max_iter+1):
		b.project({"stimA":["A"],"stimB":["B"]},
				{"A":["A","C"],"B":["B","C"],"C":["C"]})
		b_copy1 = copy.deepcopy(b)
		b_copy2 = copy.deepcopy(b)
		# in copy 1, project just A
		b_copy1.project({"stimA":["A"]},{})
		b_copy1.project({},{"A":["C"]})
		# in copy 2, project just B
		b_copy2.project({"stimB":["B"]},{})
		b_copy2.project({},{"B":["C"]})
		o = bu.overlap(b_copy1.areas["C"].winners, b_copy2.areas["C"].winners)
		results[i] = float(o)/float(k)
	return results

def merge_sim(n=100000,k=317,p=0.01,beta=0.05,max_t=50):
	b = brain.Brain(p)
	b.add_stimulus("stimA",k)
	b.add_stimulus("stimB",k)
	b.add_area("A",n,k,beta)
	b.add_area("B",n,k,beta)
	b.add_area("C",n,k,beta)

	b.project({"stimA":["A"]},{})
	b.project({"stimB":["B"]},{})
	b.project({"stimA":["A"],"stimB":["B"]},
		{"A":["A","C"],"B":["B","C"]})
	b.project({"stimA":["A"],"stimB":["B"]},
		{"A":["A","C"],"B":["B","C"],"C":["C","A","B"]})
	for i in xrange(max_t-1):
		b.project({"stimA":["A"],"stimB":["B"]},
			{"A":["A","C"],"B":["B","C"],"C":["C","A","B"]})
	return b.areas["C"].saved_w

def merge_beta_sim(n=100000,k=317,p=0.01,t=100):
	results = {}
	for beta in [0.3,0.2,0.1,0.075,0.05]:
		print ("Working on " + str(beta) + "\n")
		out = merge_sim(n,k,p,beta=beta,max_t=t)
		results[beta] = out
	return results
# UTILS FOR EVAL


def plot_project_sim(show=True, save="", show_legend=False, use_text_font=True):
	results = bu.sim_load('project_results')
	# fonts
	if(use_text_font):
		plt.rcParams['mathtext.fontset'] = 'stix'
		plt.rcParams['font.family'] = 'STIXGeneral'

	# 0.05 and 0.07 overlap almost exactly, pop 0.07
	results.pop(0.007)
	od = OrderedDict(sorted(results.items()))
	x = np.arange(100)
	print(x)
	for key,val in od.iteritems():
		plt.plot(x,val,linewidth=0.7)
	if show_legend:
		plt.legend(od.keys(), loc='upper left')
	ax = plt.axes()
	ax.set_xticks([0,10,20,50,100])
	k = 317
	plt.yticks([k,2*k,5*k,10*k,13*k],["k","2k","5k","10k","13k"])
	plt.xlabel(r'$t$')

	if not show_legend:
		for line, name in zip(ax.lines, od.keys()):
		    y = line.get_ydata()[-1]
		    ax.annotate(name, xy=(1,y), xytext=(6,0), color=line.get_color(), 
		                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
		                size=10, va="center")
	if show:
		plt.show()
	if not show and save != "":
		plt.savefig(save)

def plot_merge_sim(show=True, save="", show_legend=False, use_text_font=True):
	results = bu.sim_load('merge_betas')
	# fonts
	if(use_text_font):
		plt.rcParams['mathtext.fontset'] = 'stix'
		plt.rcParams['font.family'] = 'STIXGeneral'

	od = OrderedDict(sorted(results.items()))
	x = np.arange(101)
	for key,val in od.iteritems():
		plt.plot(x,val,linewidth=0.7)
	if show_legend:
		plt.legend(od.keys(), loc='upper left')
	ax = plt.axes()
	ax.set_xticks([0,10,20,50,100])
	k = 317
	plt.yticks([k,2*k,5*k,10*k,13*k],["k","2k","5k","10k","13k"])
	plt.xlabel(r'$t$')

	if not show_legend:
		for line, name in zip(ax.lines, od.keys()):
		    y = line.get_ydata()[-1]
		    ax.annotate(name, xy=(1,y), xytext=(6,0), color=line.get_color(), 
		                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
		                size=10, va="center")
	if show:
		plt.show()
	if not show and save != "":
		plt.savefig(save)


def plot_association(show=True, save="", use_text_font=True):
	results = bu.sim_load('association_results')
	if(use_text_font):
		plt.rcParams['mathtext.fontset'] = 'stix'
		plt.rcParams['font.family'] = 'STIXGeneral'

	od = OrderedDict(sorted(results.items()))
	plt.plot(od.keys(),od.values(),linewidth=0.7)
	ax = plt.axes()
	plt.yticks([0.1,0.2,0.3,0.4,0.5],["10%","20%","30%","40%","50%"])
	plt.xlabel(r'$t$')
	if show:
		plt.show()
	if not show and save != "":
		plt.savefig(save)

def plot_pattern_com(show=True, save="", use_text_font=True):
	results = bu.sim_load('pattern_com_iterations')
	if(use_text_font):
		plt.rcParams['mathtext.fontset'] = 'stix'
		plt.rcParams['font.family'] = 'STIXGeneral'

	od = OrderedDict(sorted(results.items()))
	plt.plot(od.keys(),od.values(),linewidth=0.7)
	ax = plt.axes()
	plt.yticks([0,0.25,0.5,0.75,1],["0%","25%","50%","75%","100%"])
	plt.xlabel(r'$t$')
	if show:
		plt.show()
	if not show and save != "":
		plt.savefig(save)

def plot_overlap(show=True, save="", use_text_font=True):
	results = bu.sim_load('overlap_results')
	if(use_text_font):
		plt.rcParams['mathtext.fontset'] = 'stix'
		plt.rcParams['font.family'] = 'STIXGeneral'

	od = OrderedDict(sorted(results.items()))
	plt.plot(od.keys(),od.values(),linewidth=0.7)
	ax = plt.axes()
	plt.xticks([0,0.2,0.4,0.6,0.8],["","20%","40%","60%","80%"])
	plt.xlabel('overlap (assemblies)')
	plt.yticks([0,0.05,0.1,0.15,0.2,0.25,0.3],["","5%","10%","15%","20%","25%","30%"])
	plt.ylabel('overlap (projections)')
	if show:
		plt.show()
	if not show and save != "":
		plt.savefig(save)

def density(n=100000,k=317,p=0.01,beta=0.05):
	b = brain.Brain(p)
	b.add_stimulus("stim",k)
	b.add_area("A",n,k,beta)
	b.project({"stim":["A"]},{})
	for i in xrange(9):
		b.project({"stim":["A"]},{"A":["A"]})
	conn = b.connectomes["A"]["A"]
	final_winners = b.areas["A"].winners
	edges = 0
	for i in final_winners:
		for j in final_winners:
			if conn[i][j] != 0:
				edges += 1
	return float(edges)/float(k**2)

def density_sim(n=100000,k=317,p=0.01,beta_values=[0,0.025,0.05,0.075,0.1]):
	results = {}
	for beta in beta_values:
		print ("Working on " + str(beta) + "\n")
		out = density(n,k,p,beta)
		results[beta] = out
	return results

def plot_density_ee(show=True,save="",use_text_font=True):
	if(use_text_font):
		plt.rcParams['mathtext.fontset'] = 'stix'
		plt.rcParams['font.family'] = 'STIXGeneral'
	od = bu.sim_load('density_results')
	plt.xlabel(r'$\beta$')
	plt.ylabel(r'assembly $p$')
	plt.plot(od.keys(),od.values(),linewidth=0.7)
	plt.plot([0,0.06],[0.01,0.01],color='red',linestyle='dashed',linewidth=0.7)
	if show:
		plt.show()
	if not show and save != "":
		plt.savefig(save)

In [7]:
import brain

import numpy as np

def larger_k(n=10000,k=100,p=0.01,beta=0.05, bigger_factor=10):
	b = brain.Brain(p, save_winners=True)
	b.add_stimulus("stim", k)
	b.add_area("A",n,k,beta)
	b.add_area("B",n,bigger_factor*k,beta)
	b.update_plasticities(area_update_map={"A":[("B", 0.8), ("A", 0.0)],
											"B":[("A", 0.8), ("B", 0.8)]})
	b.project({"stim":["A"]},{})
	t=1
	while True:
		b.project({"stim":["A"]},{"A":["A"]})
		print ("A total w is " + str(b.areas["A"].w)) 
		if (b.areas["B"].num_first_winners <= 1) and (b.areas["A"].num_first_winners <= 1):
			print ("proj(stim, A) stabilized after " + str(t) + " rounds")
			break
		t += 1
	A_after_proj = b.areas["A"].winners

	b.project({"stim":["A"]},{"A":["A","B"]})
	t=1
	while True:
		b.project({"stim":["A"]},{"A":["A","B"], "B":["B", "A"]})
		print("Num new winners in A " + str(b.areas["A"].num_first_winners))
		print("Num new winners in B " + str(b.areas["B"].num_first_winners))
		if (b.areas["B"].num_first_winners <= 1) and (b.areas["A"].num_first_winners <= 1):
			print ("recip_project(A,B) stabilized after " + str(t) + " rounds")
			break
		t += 1
	print ("Final statistics") 
	print ("A.w = " + str(b.areas["A"].w))
	print ("B.w = " + str(b.areas["B"].w))
	A_after_B = b.areas["A"].saved_winners[-1]
	o = bu.overlap(A_after_proj, A_after_B)
	print ("Overlap is " + str(o))

def turing_erase(n=50000,k=100,p=0.01,beta=0.05, r=1.0, bigger_factor=20):
	b = brain.Brain(p, save_winners=True)
	# Much smaller stimulus, similar to lower p from stimulus into A
	smaller_k = int(r*k)
	b.add_stimulus("stim", smaller_k)
	b.add_area("A",n,smaller_k,beta)
	b.add_area("B",n,bigger_factor * k,beta)
	b.add_area("C",n,bigger_factor * k,beta)
	b.update_plasticities(area_update_map={"A":[("B", 0.8),("C", 0.8), ("A", 0.0)],
											"B":[("A", 0.8), ("B", 0.8)],
											"C":[("A", 0.8), ("C", 0.8)]},
						 stim_update_map={"A":[("stim", 0.05)]})
	b.project({"stim":["A"]},{})
	t=1
	while True:
		b.project({"stim":["A"]},{"A":["A"]})
		if (b.areas["B"].num_first_winners <= 1) and (b.areas["A"].num_first_winners <= 1):
			print ("proj(stim, A) stabilized after " + str(t) + " rounds")
			break
		t += 1

	b.project({"stim":["A"]},{"A":["A","B"]})
	t=1
	while True:
		b.project({"stim":["A"]},{"A":["A","B"], "B":["B", "A"]})
		print("Num new winners in A " + str(b.areas["A"].num_first_winners))
		if (b.areas["B"].num_first_winners <= 1) and (b.areas["A"].num_first_winners <= 1):
			print ("recip_project(A,B) stabilized after " + str(t) + " rounds")
			break
		t += 1
	A_after_proj_B = b.areas["A"].winners 

	b.project({"stim":["A"]},{"A":["A","C"]})
	t=1
	while True:
		b.project({"stim":["A"]},{"A":["A","C"], "C":["C", "A"]})
		print("Num new winners in A " + str(b.areas["A"].num_first_winners))
		if (b.areas["C"].num_first_winners <= 1) and (b.areas["A"].num_first_winners <= 1):
			print ("recip_project(A,C) stabilized after " + str(t) + " rounds")
			break
		t += 1
	A_after_proj_C = b.areas["A"].winners

	# Check final conditions
	b.project({},{"A":["B"]})
	B_after_erase = b.areas["B"].saved_winners[-1]
	B_before_erase =  b.areas["B"].saved_winners[-2]
	B_overlap = bu.overlap(B_after_erase, B_before_erase)
	print("Overlap of B after erase and with y is " + str(B_overlap) + "\n")
	A_overlap = bu.overlap(A_after_proj_B,A_after_proj_C)
	print("Overlap of A after proj(B) vs after proj(C) is " + str(A_overlap) + "\n")

In [9]:
#! /usr/bin/python

from numpy.random import binomial
import heapq

n = 10000
k = 100
p = 0.01
beta = 0.05
T = 30

# stimulus to neural space is k x n
stimulus_inputs = binomial(k,p,n).astype(float)

# connectome of A (recurrent) is n x n
A_connectome = binomial(1,p,(n,n)).astype(float)

winners = []
support = set()
support_size_at_t = []
new_winners_at_t = []
# for each time step
for t in xrange(T):
	# calculate inputs into each of n neurons
	inputs = [stimulus_inputs[i] for i in xrange(n)]
	for i in winners:
		for j in xrange(n):
			inputs[j] += A_connectome[i][j]
	# identify top k winners 	
	new_winners = heapq.nlargest(k, range(len(inputs)), inputs.__getitem__)
	for i in new_winners:
		stimulus_inputs[i] *= (1+beta)
	# plasticity: for winners, for previous winners, update edge weight
	for i in winners:
		for j in new_winners:
			A_connectome[i][j] *= (1+beta)
	# update winners
	for i in new_winners:
		support.add(i)
	winners = new_winners
	support_size_at_t.append(len(support))
	if t >= 1:
		new_winners_at_t.append(support_size_at_t[-1]-support_size_at_t[-2])

NameError: ignored